# Office Hour Baseline Example Code

오피스아워 시간에 다룬 예제 코드 예제(?)입니다. 흐름을 이해할 때 실제로 코드를 돌려보면서 따라가면 이해가 더 편할 것 같아 공유하게 되었습니다. 🔥

In [1]:
from pprint import pprint

import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer

In [2]:
example_1 = {
    "context": "서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다.",
    "question": "서울의 GDP는 세계 몇 위인가?",
    "answers": {"answer_start": [24], "text": ["세계 4위"]},
    "title": "예제1",
    "id": "ex-1",
    "document_id": 1,
}

example_2 = {
    "context": "넷플릭스는 전 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 재밌다.",
    "question": "넷플릭스는 몇 개국에서 이용 중인가?",
    "answers": {"answer_start": [11], "text": ["190개국"]},
    "title": "예제2",
    "id": "ex-2",
    "document_id": 2,
}

examples = {
    key: [ex[key] for ex in [example_1, example_2]] for key in example_1.keys()
}

우선 Tokenizer가 변환하는 결과부터 살펴봅시다.

![image](./assets/01_preprocess_pipeline.png)

## 1. `prepare_train_features`

### 1-1. `tokenized_examples`

구성된 examples의 형태는 다음과 같습니다. context, question에 list형태로 문자열들이 같이 들어가고, answer에는 answer_start / text가 딕셔너리 형태로 들어가게됩니다.

In [3]:
examples

{'context': ['서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다.',
  '넷플릭스는 전 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 재밌다.'],
 'question': ['서울의 GDP는 세계 몇 위인가?', '넷플릭스는 몇 개국에서 이용 중인가?'],
 'answers': [{'answer_start': [24], 'text': ['세계 4위']},
  {'answer_start': [11], 'text': ['190개국']}],
 'title': ['예제1', '예제2'],
 'id': ['ex-1', 'ex-2'],
 'document_id': [1, 2]}

In [4]:
model_name_or_path = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

pad_on_right = tokenizer.padding_side == "right"

question_column_name = "question"
context_column_name = "context"
answer_column_name = "answers"

max_seq_length = 32
doc_stride = 16
pad_to_max_length = False

In [5]:
tokenized_examples = tokenizer(
    examples[question_column_name if pad_on_right else context_column_name],
    examples[context_column_name if pad_on_right else question_column_name],
    truncation="only_second" if pad_on_right else "only_first",
    max_length=max_seq_length,
    stride=doc_stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    # return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
    padding="max_length" if pad_to_max_length else False,
)

In [6]:
tokenized_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

오피스아워에서 설명한대로 `tokenizer`가 처리한 결과가 5개 출력되었습니다. 이는 `tokenizer`에게 주어진 parameter에 따라 다르지만 우선 저희 예제만 살펴봅시다.
- `input_ids`: `tokenizer.encode`의 결과
- `token_type_ids`: question과 context의 구분
- `attention_mask`: Encoder에게 제공될 예정이라 `[PAD]` 부분 제외하면 모두 1입니다.
- `offset_mapping`: 정답 후처리할 떄 사용
- `overflow_to_sample_mapping`: 후처리에 사용2

원래 예제에서는 두 개의 데이터를 입력하였지만, 실제 인코딩 결과는 3개가 나왔습니다. 오아에서 설명한대로 example_2는 지정된 `max_length` 길이 32를 초과하기 때문에 분절되었습니다.

In [7]:
len(tokenized_examples["input_ids"])

3

`tokenizer`의 결과는 `BatchEncoding`이라는 자료형을 가지는데, 이는 말그대로 여러 개의 `Encoding` 자료형을 가집니다. 예제에서는 `Encoding` 단위 또한 같이 살펴보겠습니다.

In [8]:
type(tokenized_examples)

transformers.tokenization_utils_base.BatchEncoding

In [9]:
te1 = tokenized_examples[0]
te2 = tokenized_examples[1]
te3 = tokenized_examples[2]

In [10]:
te1, type(te1)

(Encoding(num_tokens=28, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 tokenizers.Encoding)

이제부터 요소요소 자세히 살펴봅시다

#### `input_ids`

`input_ids`는 토큰화된 결과가 정수형 id에 매핑된 결과입니다. `Encoding`에서는 `ids`로 확인가능합니다.

In [11]:
pprint(tokenized_examples.input_ids)

[[2,
  3671,
  2079,
  9476,
  2259,
  3665,
  1077,
  18431,
  2116,
  35,
  3,
  3671,
  2073,
  3854,
  31221,
  3763,
  28674,
  18,
  3671,
  2079,
  9476,
  2259,
  3665,
  24,
  2090,
  28674,
  18,
  3],
 [2,
  23026,
  2259,
  1077,
  5620,
  27135,
  3774,
  1570,
  2179,
  2116,
  35,
  3,
  23026,
  2259,
  1537,
  3665,
  7175,
  2019,
  2226,
  3658,
  27135,
  3774,
  1570,
  28674,
  18,
  10514,
  2062,
  2839,
  2255,
  2266,
  8705,
  3],
 [2,
  23026,
  2259,
  1077,
  5620,
  27135,
  3774,
  1570,
  2179,
  2116,
  35,
  3,
  3665,
  7175,
  2019,
  2226,
  3658,
  27135,
  3774,
  1570,
  28674,
  18,
  10514,
  2062,
  2839,
  2255,
  2266,
  8705,
  7478,
  2062,
  18,
  3]]


In [12]:
te1.ids

[2,
 3671,
 2079,
 9476,
 2259,
 3665,
 1077,
 18431,
 2116,
 35,
 3,
 3671,
 2073,
 3854,
 31221,
 3763,
 28674,
 18,
 3671,
 2079,
 9476,
 2259,
 3665,
 24,
 2090,
 28674,
 18,
 3]

사실 이렇게 보면 뭐가 뭔지 알 수 없습니다. 다시 우리가 알아볼 수 있는 문자열로 돌려봅시다. 텍스트를 인코딩했으니까 다시 디코드하면 됩니다.   
여러 개의 데이터가 들어있을 때는 `tokenizer.batch_decode` 하나의 리스트만 볼 때는 `tokenizer.decode`를 사용하면 됩니다.

In [13]:
tokenizer.batch_decode(tokenized_examples.input_ids)

['[CLS] 서울의 GDP는 세계 몇 위인가? [SEP] 서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다. [SEP]',
 '[CLS] 넷플릭스는 몇 개국에서 이용 중인가? [SEP] 넷플릭스는 전 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 [SEP]',
 '[CLS] 넷플릭스는 몇 개국에서 이용 중인가? [SEP] 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 재밌다. [SEP]']

In [14]:
tokenizer.decode(te1.ids)

'[CLS] 서울의 GDP는 세계 몇 위인가? [SEP] 서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다. [SEP]'

아까 `stride=16`으로 제공했는데 잘 분절되었는지 볼까요?

In [15]:
sep_index = te3.ids.index(tokenizer.sep_token_id) + 1

print(te2.ids[-(doc_stride + 1):-1])
print(te3.ids[sep_index:sep_index + doc_stride])

[3665, 7175, 2019, 2226, 3658, 27135, 3774, 1570, 28674, 18, 10514, 2062, 2839, 2255, 2266, 8705]
[3665, 7175, 2019, 2226, 3658, 27135, 3774, 1570, 28674, 18, 10514, 2062, 2839, 2255, 2266, 8705]


잘 분절된 것 같네요. 나머지 4개의 feature도 살펴봅시다.

#### `token_type_ids`

두 문장 (우리의 경우 question & context)를 넣을 때 두 문장을 구분할 수 있도록 제공되는 자료형입니다. 말로 봐서는 뭔지 모르겠으니 다시 봅시다.
`Encoding`에서는 `type_ids`로 호출 가능합니다.

In [16]:
print(te1.type_ids)
print(te1.ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2, 3671, 2079, 9476, 2259, 3665, 1077, 18431, 2116, 35, 3, 3671, 2073, 3854, 31221, 3763, 28674, 18, 3671, 2079, 9476, 2259, 3665, 24, 2090, 28674, 18, 3]


실제로 0과 1이 각각 Question과 Context 위치를 담고 있는지 살펴볼까요? (`target = te2`나 `te3`으로 바꿔놓고도 해보세요)

In [17]:
target = te1

for token_type_id in np.unique(target.type_ids):
    mask = target.type_ids == token_type_id
    _ids = np.array(target.ids)[mask].tolist()
    
    print(f"`token_type_id` = {token_type_id}")
    print(f"Raw `input_ids` = {_ids}")
    print(f"Decoded Result  = {tokenizer.decode(_ids)}", end="\n\n")

`token_type_id` = 0
Raw `input_ids` = [2, 3671, 2079, 9476, 2259, 3665, 1077, 18431, 2116, 35, 3]
Decoded Result  = [CLS] 서울의 GDP는 세계 몇 위인가? [SEP]

`token_type_id` = 1
Raw `input_ids` = [3671, 2073, 3854, 31221, 3763, 28674, 18, 3671, 2079, 9476, 2259, 3665, 24, 2090, 28674, 18, 3]
Decoded Result  = 서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다. [SEP]



#### `attention_mask`

사실 이건 살펴볼 것도 없이 전부 1이긴합니다.

In [18]:
for attn in tokenized_examples["attention_mask"]:
    print(attn)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#### `offset_mapping`

사실 제일 직관적이지 않으면서 해석이 난해한 두 가지가 바로 `offset_mapping`과 `overflow_to_sample_mapping`인데요, 눈으로 보는 게 이해가 조금 더 쉽습니다.   
`offset_mapping`은 오아에서 설명했듯이 실제 토큰이 **원래 original text에서 어느 위치를 가지는지 표시**합니다.

![image](./assets/02_offset_mapping.png)

In [19]:
target_index = 0
target = tokenized_examples[target_index]

orig_texts = [examples["question"][target_index],
              examples["context"][target_index]]
print(f"Original Text: {orig_texts}", end="\n\n")
for token, type_id, offset in zip(target.tokens, target.type_ids, target.offsets):
    print(f"Token                     : {token}")
    print(f"Offset                    : {offset}")

    s, e = offset
    print(f"Text retreived with offset: {orig_texts[type_id][s:e]}", end="\n\n")

Original Text: ['서울의 GDP는 세계 몇 위인가?', '서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다.']

Token                     : [CLS]
Offset                    : (0, 0)
Text retreived with offset: 

Token                     : 서울
Offset                    : (0, 2)
Text retreived with offset: 서울

Token                     : ##의
Offset                    : (2, 3)
Text retreived with offset: 의

Token                     : GDP
Offset                    : (4, 7)
Text retreived with offset: GDP

Token                     : ##는
Offset                    : (7, 8)
Text retreived with offset: 는

Token                     : 세계
Offset                    : (9, 11)
Text retreived with offset: 세계

Token                     : 몇
Offset                    : (12, 13)
Text retreived with offset: 몇

Token                     : 위인
Offset                    : (14, 16)
Text retreived with offset: 위인

Token                     : ##가
Offset                    : (16, 17)
Text retreived with offset: 가

Token                     : ?
Offse

눈으로 보니 제법 직관적이죠? 이게 왜 필요한지는 눈치가 백단이신분들은 눈치 챘겠지만, `answer`의 토큰위치를 뽑아오기 위함입니다. 현재 원본 데이터의 `answer` 내에는 answer의 원본텍스트 내에서의 위치만 가지고 있어요. 얘를 토큰화된 결과의 위치를 뽑아주려면 `offset_mapping`이 필요합니다!

#### `overflow_to_sample_mapping`

`tokenizer`의 마지막 결과인 `overflow_to_sample_mapping`을 살펴봅시다. 오아에서 언급했듯, 해당 feature는 실제로 주어진 입력 데이터가 길이가 길어져 쪼개진 예제2번과 같은 문장이 있을 때 원본데이터에서 몇 번째 데이터인지 알려주는 정보입니다.

이게 왜 필요할까요? 우리의 예제에는 question, context만 있는 것이 아니라, answer도 있는데 우리가 answer는 `tokenizer`에 따로 태워주지 않기 때문에 얘는 어리둥절하게 자기랑 한쌍이었던 question/context가 쪼개진지 모르고 있기 때문입니다. 현재 우리의 extraction-based MRC에서는 정답 텍스트가 그대로 필요하기보다는, 토큰화된 결과에서 정답이 "몇 번째부터 몇 번쨰"에 위치하는지가 중요하기 때문입니다.

In [20]:
tokenized_examples["overflow_to_sample_mapping"]

[0, 1, 1]

In [21]:
for idx, ovf in enumerate(tokenized_examples["overflow_to_sample_mapping"]):
    _tkd = tokenizer.decode(tokenized_examples["input_ids"][idx])
    print(f"토큰화된 결과 내에서의 index        : {idx}, {_tkd}")
    print(f"해당 데이터의 원본데이터 내에서의 index: {ovf}, {examples['question'][ovf]} {examples['context'][ovf]}", end="\n\n")

토큰화된 결과 내에서의 index        : 0, [CLS] 서울의 GDP는 세계 몇 위인가? [SEP] 서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다. [SEP]
해당 데이터의 원본데이터 내에서의 index: 0, 서울의 GDP는 세계 몇 위인가? 서울은 국제적인 도시이다. 서울의 GDP는 세계 4위이다.

토큰화된 결과 내에서의 index        : 1, [CLS] 넷플릭스는 몇 개국에서 이용 중인가? [SEP] 넷플릭스는 전 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 [SEP]
해당 데이터의 원본데이터 내에서의 index: 1, 넷플릭스는 몇 개국에서 이용 중인가? 넷플릭스는 전 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 재밌다.

토큰화된 결과 내에서의 index        : 2, [CLS] 넷플릭스는 몇 개국에서 이용 중인가? [SEP] 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 재밌다. [SEP]
해당 데이터의 원본데이터 내에서의 index: 1, 넷플릭스는 몇 개국에서 이용 중인가? 넷플릭스는 전 세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 재밌다.



이제야 tokenizer의 결과가 다 이해되는 것 같네요. 여기까지도 오랜 시간이 걸렸는데 사실 여기가 끝이 아닌거 아시죠...? 아직 모델에게 제공할 수는 없는 상태의 데이터입니다. 베이스라인에서 `prepare_train_features`의 한 단계 밖에 오지 않았어요. 토큰화된 결과를 어떻게 QA Task에 맞춰서 변경하는지 살펴봅시다.

### 1-2. Preprocess to QA Task

우선 코드를 살펴봅시다.

In [22]:
# Train preprocessing / 전처리를 진행합니다.
def prepare_train_features(examples):
    ###############################
    ####### 지금 여기까지 본거임 #######
    ###############################
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        # return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
        padding="max_length" if pad_to_max_length else False,
    )
    ###############################
    ###############################
    ###############################


    # 여기부터 봅시다. 코드만 보려고 여기선 주석을 제거했어요
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

드디어 `sample_mapping`과 `offset_mapping`이 무엇을 하는지 이해했네요.

In [23]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
offset_mapping = tokenized_examples.pop("offset_mapping")

tokenized_examples["start_positions"] = []
tokenized_examples["end_positions"] = []

루프를 `offset_mapping`에 대해 돌려주네요. 아까 언급했듯이 정답데이터를 같이 활용해야 하기 때문입니다. 아래 코드는 의도적으로 `break를` 걸어두었는데요, 중간 변수를 살펴보시라고 걸어두었습니다. 일단은 0번 데이터에 대해서 걸어두었어요

In [24]:
meomchow = 0
for i, offsets in enumerate(offset_mapping):
    input_ids = tokenized_examples["input_ids"][i]
    cls_index = input_ids.index(tokenizer.cls_token_id)

    sequence_ids = tokenized_examples.sequence_ids(i)

    sample_index = sample_mapping[i]
    answers = examples[answer_column_name][sample_index]

    if len(answers["answer_start"]) == 0:
        tokenized_examples["start_positions"].append(cls_index)
        tokenized_examples["end_positions"].append(cls_index)
    else:
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])

        token_start_index = 0
        while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
            token_start_index += 1

        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
            token_end_index -= 1

        if not (
            offsets[token_start_index][0] <= start_char
            and offsets[token_end_index][1] >= end_char
        ):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            while (
                token_start_index < len(offsets)
                and offsets[token_start_index][0] <= start_char
            ):
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    if i == meomchow:
        break

위에서부터 순차적으로 살펴봅시다.

In [25]:
tokenized_examples["start_positions"] = []
tokenized_examples["end_positions"] = []

input_ids = tokenized_examples["input_ids"][i]
cls_index = input_ids.index(tokenizer.cls_token_id)

sequence_ids = tokenized_examples.sequence_ids(i)

sample_index = sample_mapping[i]
answers = examples[answer_column_name][sample_index]

In [26]:
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [27]:
answers

{'answer_start': [24], 'text': ['세계 4위']}

다행히 여기 있는 변수는 모두 아는 변수네요. 복기하면
- `input_ids`: 토큰화된 결과
- `cls_index`: `cls` 토큰의 위치입니다. 우리는 0번 밖에 나올 게 없어요
- `sequence_ids`: 아까 살펴본 `token_type_ids`와 정확히 같은 기능인데 차이점은 딱 하나: special_token([CLS], [SEP]) 자리에 0, 1이 아닌 `None`을 밀어넣어줍니다.
- `sample_mapping`: 원본 텍스트 내에서의 토큰의 위치입니다
- `answers`: 원본 입력에서 `answer` 뽑아왔습니다.

In [28]:
if len(answers["answer_start"]) == 0:
    tokenized_examples["start_positions"].append(cls_index)
    tokenized_examples["end_positions"].append(cls_index)

첫 번째 분기인데요, 조건이 `len(answers["answer_start"]) == 0`이네요. 정답이 없는 경우는 그냥 `cls_index`를 정답으로 밀어넣도록 설계되어있네요. edge case의 경우니까 else문으로 넘어갑시다.

In [29]:
# 마음 속에 `else`가 여기 있었다고 생각하세요

start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

print(start_char, end_char)
print(f"Extracted answer : {examples['context'][meomchow][start_char:end_char]}")
print(f"Actual answer    : {examples['answers'][meomchow]['text'][0]}")

24 29
Extracted answer : 세계 4위
Actual answer    : 세계 4위


원본 텍스트 내에서의 정답 위치를 뽑아왔습니다. 24번째부터 29번째까지가 정답 단어에 해당하는 위치입니다. end_char는 단순히 텍스트 길이로 뽑아옵니다.

In [30]:
print(pad_on_right)

True


In [31]:
token_start_index = 0
while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
    token_start_index += 1

token_end_index = len(input_ids) - 1
while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
    token_end_index -= 1

해당 부분은 **전체 토큰 결과 중에 context의 시작점과 끝점**을 찾아줍니다. 여기서 자주 질문이 들어오는 부분이 `pad_on_right`인데요, 얘는 context를 오른쪽에다 밀어넣을 것인지 왼쪽에 밀어넣을 것인지 입니다. 코드를 바꿔 쓰면
```python
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

token_end_index = len(input_ids) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1
``` 

In [32]:
print(f"`sequence_ids` = {sequence_ids}")

print(f"Extracted context {sequence_ids[token_start_index:token_end_index]}")

`sequence_ids` = [None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]
Extracted context [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


context만 잘 뽑힌 것을 알 수 있습니다.

In [33]:
if not (
    offsets[token_start_index][0] <= start_char
    and offsets[token_end_index][1] >= end_char
):
    tokenized_examples["start_positions"].append(cls_index)
    tokenized_examples["end_positions"].append(cls_index)

첫 번째 분기부터 살펴봅시다. `offsets`는 계속 언급했지만 원본 텍스트 내에서의 토큰이 가지는 위치입니다.
![image](./assets/02_offset_mapping.png)

if문 내에서 비교되는 데이터들은 모두 **원본 데이터 내에서의 위치를 비교**하는 것입니다.

In [34]:
# token_start_index = context가 시작하는 token화 결과 내의 위치
print(token_start_index)

# offsets[token_start_index] = context가 실제 텍스트 내에서 시작하는 위치.
# 당연히 context가 시작하는 위치니까 0이 나와야 정상입니다.
print(offsets[token_start_index][0])

# 이 값을 start_char와 비교합니다.
print(start_char)
print(offsets[token_start_index][0] <= start_char)

11
0
24
True


만약 정답이 context 내에 잘 위치해있다고 생각해봅시다.   
정답의 시작은 당연히 context의 시작보다 뒤에 나와야하기 때문입니다. end_char를 비교하는 것도 같은 맥락입니다. 실제로 context가 끝나는 위치보다 항상 end_char가 작도록 비교합니다.
여기에 `not`을 붙여주니까 **정답이 context 내에 잘 위치하지 못하는 경우**를 잡아주는 것이네요! 여기도 edge case를 잡아주는 것이라 볼 수 있겠습니다.

이 경우에도 마찬가지로 `cls_index`를 정답으로 예측하도록 밀어줍니다.

만약 정답이 잘 context 내에 들어있다면 if를 통과하고 else로 넘어오겠네요. 그 코드가 아래에 해당합니다.

In [35]:
# 이번엔 마음 속의 if 가 있었다고 칩시다.
while (
    token_start_index < len(offsets)
    and offsets[token_start_index][0] <= start_char
):
    token_start_index += 1
tokenized_examples["start_positions"].append(token_start_index - 1)

while offsets[token_end_index][1] >= end_char:
    token_end_index -= 1
tokenized_examples["end_positions"].append(token_end_index + 1)

token_start_index는 context의 시작점이고 len(offsets)는 전체 token 개수에 해당합니다.
`offsets[token_start_index[0]] <= start_char`는 아까도 확인했듯이 정답의 시작 위치가 context보다 뒤에 정상적으로 안착한 경우를 뜻합니다.

여기서 token_start_index를 계속 `+= 1`씩 증가시켜주는데요, 이제부터는 `token_start_index`가 context의 시작위치가 아닌 정답의 토큰시작 위치를 알리는 변수로 사용됩니다.
이 loop가 언제 깨질까요? `token_start_index`는 계속 1씩 증가할테니
1. 실제 정답 위치보다 앞서나갈 때
2. 혹은 정답을 못 찾아서 결국 전체 token 길이보다 커져버릴 때

위의 경우는 정답을 찾아버렸으니 +1하는 과정에서 정답 뒤로 넘어간 경우겠죠? 그렇기 때문에 실제로 `start_positions` 위치에는 -1을 해준 값을 넣어주게 됩니다.

아래의 `end_position` 찾는 것도 마찬가지입니다.

Sanity check를 해봅시다. 해당 token position이 실제로 정답을 잘 바라보고 있는지 확인해봅시다.

In [36]:
st, et = tokenized_examples["start_positions"][meomchow], tokenized_examples["end_positions"][meomchow]

In [37]:
answer_token = tokenized_examples.input_ids[meomchow][st:et + 1]
tokenizer.decode(answer_token)

'세계 4위'

휴 다행히 결과가 잘 나오는군요. `meomchow` 변수를 1이나 2로 바꿔서 또 test 해보세요! 마지막으로 실제 `datasets.map`을 걸고난 결과를 비교하는 것입니다. `num_rows`와 `features` 정보가 일부 바뀐 것을 확인해보세요.

In [38]:
train_dataset = Dataset.from_dict(examples)
print("Before Preprocess")
print(train_dataset)
train_dataset = train_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=train_dataset.column_names,
)
print("After Preprocess")
print(train_dataset)

Before Preprocess
Dataset({
    features: ['context', 'question', 'answers', 'title', 'id', 'document_id'],
    num_rows: 2
})
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

After Preprocess
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 3
})


## 2. `prepare_validation_features`

오아에서 시간이 부족해 다루지 않은 부분인데요, 간단하게 살펴보는 코드를 넣어보았습니다. Validation의 경우 start, end token 위치를 맞출 필요가 없이 정답 text만 잘 가져다주면 되다보니 정답을 처리하는 일은 따로 없습니다. 대신에 `tokenizer`가 `max_seq_length` 로직으로 **잘라 버린 데이터들에 대해 정답데이터를 매칭시켜주기 위해 `example_id`만 잘 넣어줍니다.**

In [39]:
tokenized_examples = tokenizer(
    examples[question_column_name if pad_on_right else context_column_name],
    examples[context_column_name if pad_on_right else question_column_name],
    truncation="only_second" if pad_on_right else "only_first",
    max_length=max_seq_length,
    stride=doc_stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    # return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
    padding="max_length" if pad_to_max_length else False,
)

# 길이가 긴 context가 등장할 경우 truncate를 진행해야하므로, 해당 데이터셋을 찾을 수 있도록 mapping 가능한 값이 필요합니다.
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

# evaluation을 위해, prediction을 context의 substring으로 변환해야합니다.
# corresponding example_id를 유지하고 offset mappings을 저장해야합니다.
tokenized_examples["example_id"] = []

for i in range(len(tokenized_examples["input_ids"])):
    # sequence id를 설정합니다 (to know what is the context and what is the question).
    sequence_ids = tokenized_examples.sequence_ids(i)
    context_index = 1 if pad_on_right else 0

    # 하나의 example이 여러개의 span을 가질 수 있습니다.
    sample_index = sample_mapping[i]
    tokenized_examples["example_id"].append(examples["id"][sample_index])

    # Set to None the offset_mapping을 None으로 설정해서 token position이 context의 일부인지 쉽게 판별 할 수 있습니다.
    tokenized_examples["offset_mapping"][i] = [
        (o if sequence_ids[k] == context_index else None)
        for k, o in enumerate(tokenized_examples["offset_mapping"][i])
    ]

In [40]:
tokenized_examples["example_id"]

['ex-1', 'ex-2', 'ex-2']

In [41]:
tokenized_examples["offset_mapping"][0]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 (0, 2),
 (2, 3),
 (4, 6),
 (6, 8),
 (9, 11),
 (11, 13),
 (13, 14),
 (15, 17),
 (17, 18),
 (19, 22),
 (22, 23),
 (24, 26),
 (27, 28),
 (28, 29),
 (29, 31),
 (31, 32),
 None]

함수화해서 실제 변형되는 케이스를 살펴봅시다.

In [42]:
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        # return_token_type_ids=False, # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
        padding="max_length" if pad_to_max_length else False,
    )

    # 길이가 긴 context가 등장할 경우 truncate를 진행해야하므로, 해당 데이터셋을 찾을 수 있도록 mapping 가능한 값이 필요합니다.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # evaluation을 위해, prediction을 context의 substring으로 변환해야합니다.
    # corresponding example_id를 유지하고 offset mappings을 저장해야합니다.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # sequence id를 설정합니다 (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # 하나의 example이 여러개의 span을 가질 수 있습니다.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping을 None으로 설정해서 token position이 context의 일부인지 쉽게 판별 할 수 있습니다.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
    return tokenized_examples

In [43]:
eval_examples = Dataset.from_dict(examples)
print("**** Before Preprocess ****")
print(eval_examples)
eval_dataset = eval_examples.map(
    prepare_validation_features,
    batched=True,
    remove_columns=eval_examples.column_names,
)
print("**** After Preprocess ****")
print(eval_dataset)

**** Before Preprocess ****
Dataset({
    features: ['context', 'question', 'answers', 'title', 'id', 'document_id'],
    num_rows: 2
})


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

**** After Preprocess ****
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 3
})


## 3. `postprocess_qa_predictions`

후처리 결과도 상당히 중요한데 매번 시간이 부족해서 살펴보지 못해서 짧게 넣어봤습니다. 계속 제일 중요한 부분은 쪼개진 데이터들을 어떻게 합쳐오느냐에 있어요. 두 개로 분절되는 예제2번에 주의해주세요  
‼️ 주의: 아래 코드는 저희가 제공한 베이스라인에서 차용한 것은 맞으나 최소한의 세팅을 하기 위해 일부 코드가 변형되어 있습니다. 굉장히 사소하게 차이가 있기 때문에 주의하세요!

In [44]:
from transformers import AutoConfig, AutoModelForQuestionAnswering, EvalPrediction, TrainingArguments
import evaluate

from trainer_qa import QuestionAnsweringTrainer
from utils_qa import postprocess_qa_predictions

training_args = TrainingArguments(do_eval=True, output_dir=".")

In [45]:
max_answer_length = 30
metric = evaluate.load("squad")

def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

def post_processing_function(examples, features, predictions, training_args):
    # Post-processing: start logits과 end logits을 original context의 정답과 match시킵니다.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        max_answer_length=max_answer_length,
        output_dir=None,
    )
    # Metric을 구할 수 있도록 Format을 맞춰줍니다.
    formatted_predictions = [
        {"id": k, "prediction_text": v} for k, v in predictions.items()
    ]
    if training_args.do_predict:
        return formatted_predictions

    elif training_args.do_eval:
        references = [
            {"id": ex["id"], "answers": ex[answer_column_name]}
            for ex in eval_examples
        ]
        return EvalPrediction(
            predictions=formatted_predictions, label_ids=references
        )

In [46]:
config = AutoConfig.from_pretrained(model_name_or_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_name_or_path, config=config)

trainer = QuestionAnsweringTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    eval_examples=eval_examples,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

자 위에까지는 간단한 세팅이었구요 아래는 `QuestionAnsweringTrainer.evaluate` 메소드를 일부 가져온 것입니다. 여기서 주안점은 `output`에 어떤 게 어떻게 들어오는지 보는 것이에요.

In [47]:
self = trainer

eval_dataset = self.eval_dataset
eval_dataloader = self.get_eval_dataloader(eval_dataset)
eval_examples = self.eval_examples

ignore_keys = None

# 일시적으로 metric computation를 불가능하게 한 상태이며, 해당 코드에서는 loop 내에서 metric 계산을 수행합니다.
compute_metrics = self.compute_metrics
self.compute_metrics = None
try:
    output = self.prediction_loop(
        eval_dataloader,
        description="Evaluation",
        # metric이 없으면 예측값을 모으는 이유가 없으므로 아래의 코드를 따르게 됩니다.
        # self.args.prediction_loss_only
        prediction_loss_only=True if compute_metrics is None else None,
        ignore_keys=ignore_keys,
    )
finally:
    self.compute_metrics = compute_metrics
    
if isinstance(eval_dataset, Dataset):
    eval_dataset.set_format(
        type=eval_dataset.format["type"],
        columns=list(eval_dataset.features.keys()),
    )

/Users/daehyuncho/anaconda3/envs/mrc/lib/python3.10/site-packages/transformers/trainer_pt_utils.py:411: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
type(output), output

(transformers.trainer_utils.EvalLoopOutput,
 EvalLoopOutput(predictions=(array([[-2.27704510e-01,  2.16861933e-01,  9.41800296e-01,
          4.41306651e-01,  2.32540250e-01,  8.80979180e-01,
         -4.31050867e-01, -1.55686215e-01,  6.72641158e-01,
          3.40887159e-02,  1.96681008e-01,  3.32069993e-01,
          5.18067956e-01,  1.04460359e+00,  4.56062078e-01,
          2.61725456e-01,  5.75585783e-01,  2.71887213e-01,
          1.98809683e-01,  8.62057626e-01,  3.42595547e-01,
          5.80603421e-01,  7.32531667e-01,  7.42925227e-01,
          1.18085706e+00,  6.21984661e-01,  2.02956215e-01,
          1.96060240e-01, -1.35519832e-01, -4.08501625e-02,
         -5.07392883e-01, -3.61381412e-01],
        [-1.06058374e-01,  5.95110178e-01,  4.28310269e-03,
         -5.18237948e-01, -5.51881194e-01,  3.06386590e-01,
          1.50878072e-01,  5.29339790e-01,  3.29761326e-01,
          7.90486276e-01,  5.05057216e-01,  1.51467636e-01,
          5.33966959e-01,  1.93225041e-01,  

`EvalLoopOutput`은 왜인지 모르겠는데 `__dict__`가 없어서 눈으로 보고 가져오거나 `dir`함수로 지원하는 attribute를 확인할 수 있습니다.

In [49]:
[i for i in dir(output) if not i.startswith('_')]

['count', 'index', 'label_ids', 'metrics', 'num_samples', 'predictions']

중요한건 `predictions`니까 `predictions`만 봅시다.

In [50]:
type(output.predictions), output.predictions

(tuple,
 (array([[-2.27704510e-01,  2.16861933e-01,  9.41800296e-01,
           4.41306651e-01,  2.32540250e-01,  8.80979180e-01,
          -4.31050867e-01, -1.55686215e-01,  6.72641158e-01,
           3.40887159e-02,  1.96681008e-01,  3.32069993e-01,
           5.18067956e-01,  1.04460359e+00,  4.56062078e-01,
           2.61725456e-01,  5.75585783e-01,  2.71887213e-01,
           1.98809683e-01,  8.62057626e-01,  3.42595547e-01,
           5.80603421e-01,  7.32531667e-01,  7.42925227e-01,
           1.18085706e+00,  6.21984661e-01,  2.02956215e-01,
           1.96060240e-01, -1.35519832e-01, -4.08501625e-02,
          -5.07392883e-01, -3.61381412e-01],
         [-1.06058374e-01,  5.95110178e-01,  4.28310269e-03,
          -5.18237948e-01, -5.51881194e-01,  3.06386590e-01,
           1.50878072e-01,  5.29339790e-01,  3.29761326e-01,
           7.90486276e-01,  5.05057216e-01,  1.51467636e-01,
           5.33966959e-01,  1.93225041e-01,  8.24892163e-01,
           5.75716496e-01,  5.72

잘보면 predictions가 일반적인 텐서가 아니라 튜플형태로 제공되고 길이가 2라는 점입니다. Extraction-based MRC의 특성을 생각해보면 Start지점과 End지점을 예측해야하니 어찌보면 당연합니다. loss는 애초에 datasets에다가 토큰 위치를 넣어줬으니 계산하는 것은 BCE로 쉽게 계산했을 것입니다.

다만 실제로 정답 텍스트를 추출해서 loss가 아닌 다른 metric을 측정하는 것은 또 다른 문제인데요, 어떻게 처리했는지 확인해봅시다.

In [51]:
if self.post_process_function is not None and self.compute_metrics is not None:
    eval_preds = self.post_process_function(
        eval_examples, eval_dataset, output.predictions, self.args
    )
    metrics = self.compute_metrics(eval_preds)

  0%|          | 0/2 [00:00<?, ?it/s]

여기서 사용된 `self.post_process_function`은 베이스라인 `train.py` 내에 있는 `post_processing_function`을 사용하는 것인데요, 이를 사용하는 것보다도 더 중요한 건 이 안에서 사용되는 `utils_qa.py` 내의 `postprocess_qa_predictions` 함수입니다. 일단 결과부터 살펴봅시다.

In [52]:
eval_preds.label_ids

[{'id': 'ex-1', 'answers': {'answer_start': [24], 'text': ['세계 4위']}},
 {'id': 'ex-2', 'answers': {'answer_start': [11], 'text': ['190개국']}}]

In [53]:
eval_preds.predictions

[{'id': 'ex-1', 'prediction_text': '국제적인 도시이다. 서울의'},
 {'id': 'ex-2', 'prediction_text': '세계 190개국 이상에서 이용 중이다. 스물다섯스물하나 재밌'}]

데이터는 3개를 제공했는데 잘 고유 id 개수에 맞춰서 결과를 변환해줬네요. 이 모든 게 `postprocess_qa_predictions` 안에서 일어나는 일입니다. 세세하게 모든 과정을 다 보지는 않는 대신에 이 함수에 제공되는 입력값들을 알고 있으면 어느 정도 함수 내의 흐름을 이해하는데 도움이 되기 때문에 작성해보았습니다. 어떤 자료형이 들어가는지 이해하면 저희가 `utils_qa.py` 내에 자세하게 한글로 주석을 남겨놓았기 때문에 이해하는데 조금이나마 더 도움이 될 것 같습니다!

```python
def postprocess_qa_predictions(
    examples,                                   # <- eval_examples
    features,                                   # <- eval_dataset
    predictions: Tuple[np.ndarray, np.ndarray], # <- output.predictions
    version_2_with_negative: bool = False,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    is_world_process_zero: bool = True,
):
```

`eval_examples`와 `eval_dataset`의 차이는 전처리 차이입니다. `eval_examples`는 코드를 확인해보면 전처리가 되기 전의 `dataset["validation"]`을 입력으로 받습니다. `eval_dataset`은 `preprocess_validation_features` 함수를 거친 데이터셋입니다.

In [54]:
eval_examples

Dataset({
    features: ['context', 'question', 'answers', 'title', 'id', 'document_id'],
    num_rows: 2
})

In [55]:
eval_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 3
})

`output.predictions`는 start_token 확률과 end_token 확률을 계산할 수 있는 logit값을 포함합니다. 첫번째 행렬은 start_token, 두번째 행렬은 end_token의 값입니다.

In [56]:
output.predictions

(array([[-2.27704510e-01,  2.16861933e-01,  9.41800296e-01,
          4.41306651e-01,  2.32540250e-01,  8.80979180e-01,
         -4.31050867e-01, -1.55686215e-01,  6.72641158e-01,
          3.40887159e-02,  1.96681008e-01,  3.32069993e-01,
          5.18067956e-01,  1.04460359e+00,  4.56062078e-01,
          2.61725456e-01,  5.75585783e-01,  2.71887213e-01,
          1.98809683e-01,  8.62057626e-01,  3.42595547e-01,
          5.80603421e-01,  7.32531667e-01,  7.42925227e-01,
          1.18085706e+00,  6.21984661e-01,  2.02956215e-01,
          1.96060240e-01, -1.35519832e-01, -4.08501625e-02,
         -5.07392883e-01, -3.61381412e-01],
        [-1.06058374e-01,  5.95110178e-01,  4.28310269e-03,
         -5.18237948e-01, -5.51881194e-01,  3.06386590e-01,
          1.50878072e-01,  5.29339790e-01,  3.29761326e-01,
          7.90486276e-01,  5.05057216e-01,  1.51467636e-01,
          5.33966959e-01,  1.93225041e-01,  8.24892163e-01,
          5.75716496e-01,  5.72912395e-01,  6.03487007e-

`postprocess_qa_predictions`는 로직 자체도 그렇게 복잡하지 않으나 어떤 데이터가 들어가는지 몰라서 따라가기가 힘든데요, 여기서 시작하시면 조금 더 편하실거라 예상합니다 :) 

LLM의 시대로 접어들면서 모델의 규모가 커진 것도 중요하지만, 저는 그만큼 좋은 데이터를 입력으로 주는 것이 굉장히 중요하다고 생각합니다. 너무 조급하게 생각하지 마시고 이번 MRC에서는 Huggingface 사용방법과 여기에 데이터를 전처리헤서 넣어주는 방법들에 대해 많이 공부해보시면 좋을 것 같습니다. 그럼 20000


### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License